In [1]:
import numpy as np

## Source 

https://nathanrooy.github.io/posts/2018-03-22/word2vec-from-scratch-with-python-and-numpy/

## Code

In [8]:
#------------------------------------------------------------------------------+
#
#   Nathan A. Rooy
#   Simple word2vec from scratch with Python
#   2018-FEB
#
#------------------------------------------------------------------------------+

#--- IMPORT DEPENDENCIES ------------------------------------------------------+

import numpy as np
import re
from collections import defaultdict

#--- CONSTANTS ----------------------------------------------------------------+


class word2vec():
    def __init__ (self):
        self.n = settings['n']
        self.eta = settings['learning_rate']
        self.epochs = settings['epochs']
        self.window = settings['window_size']
        pass
    
    
    # GENERATE TRAINING DATA
    def generate_training_data(self, settings, corpus):

        # GENERATE WORD COUNTS
        word_counts = defaultdict(int)
        for row in corpus:
            for word in row:
                word_counts[word] += 1

        self.v_count = len(word_counts.keys())

        # GENERATE LOOKUP DICTIONARIES
        self.words_list = sorted(list(word_counts.keys()),reverse=False)
        self.word_index = dict((word, i) for i, word in enumerate(self.words_list))
        self.index_word = dict((i, word) for i, word in enumerate(self.words_list))

        training_data = []
        # CYCLE THROUGH EACH SENTENCE IN CORPUS
        for sentence in corpus:
            sent_len = len(sentence)

            # CYCLE THROUGH EACH WORD IN SENTENCE
            for i, word in enumerate(sentence):
                
                #w_target  = sentence[i]
                w_target = self.word2onehot(sentence[i])

                # CYCLE THROUGH CONTEXT WINDOW
                w_context = []
                for j in range(i-self.window, i+self.window+1):
                    if j!=i and j<=sent_len-1 and j>=0:
                        w_context.append(self.word2onehot(sentence[j]))
                training_data.append([w_target, w_context])
        return np.array(training_data)


    # SOFTMAX ACTIVATION FUNCTION
    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)


    # CONVERT WORD TO ONE HOT ENCODING
    def word2onehot(self, word):
        word_vec = [0 for i in range(0, self.v_count)]
        word_index = self.word_index[word]
        word_vec[word_index] = 1
        return word_vec


    # FORWARD PASS
    def forward_pass(self, x):
        h = np.dot(self.w1.T, x)
        u = np.dot(self.w2.T, h)
        y_c = self.softmax(u)
        return y_c, h, u
                

    # BACKPROPAGATION
    def backprop(self, e, h, x):
        dl_dw2 = np.outer(h, e)  
        dl_dw1 = np.outer(x, np.dot(self.w2, e.T))

        # UPDATE WEIGHTS
        self.w1 = self.w1 - (self.eta * dl_dw1)
        self.w2 = self.w2 - (self.eta * dl_dw2)
        pass


    # TRAIN W2V model
    def train(self, training_data):
        # INITIALIZE WEIGHT MATRICES
        self.w1 = np.random.uniform(-0.8, 0.8, (self.v_count, self.n))     # context matrix
        self.w2 = np.random.uniform(-0.8, 0.8, (self.n, self.v_count))     # embedding matrix
        
        # CYCLE THROUGH EACH EPOCH
        for i in range(0, self.epochs):

            self.loss = 0

            # CYCLE THROUGH EACH TRAINING SAMPLE
            for w_t, w_c in training_data:

                # FORWARD PASS
                y_pred, h, u = self.forward_pass(w_t)
                
                # CALCULATE ERROR
                EI = np.sum([np.subtract(y_pred, word) for word in w_c], axis=0)

                # BACKPROPAGATION
                self.backprop(EI, h, w_t)

                # CALCULATE LOSS
                self.loss += -np.sum([u[word.index(1)] for word in w_c]) + len(w_c) * np.log(np.sum(np.exp(u)))
                self.loss += -2*np.log(len(w_c)) -np.sum([u[word.index(1)] for word in w_c]) + (len(w_c) * np.log(np.sum(np.exp(u))))
                
            print('EPOCH:',i, 'LOSS:', self.loss)
        pass


    # input a word, returns a vector (if available)
    def word_vec(self, word):
        w_index = self.word_index[word]
        v_w = self.w1[w_index]
        return v_w


    # input a vector, returns nearest word(s)
    def vec_sim(self, vec, top_n):

        # CYCLE THROUGH VOCAB
        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(vec, v_w2)
            theta_den = np.linalg.norm(vec) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda word, sim: sim, reverse=True)

        for word, sim in words_sorted[:top_n]:
            print(word, sim)
            
        pass

    # input word, returns top [n] most similar words
    def word_sim(self, word, top_n):
        
        w1_index = self.word_index[word]
        v_w1 = self.w1[w1_index]

        # CYCLE THROUGH VOCAB
        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(v_w1, v_w2)
            theta_den = np.linalg.norm(v_w1) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda word, sim:sim, reverse=True)

        for word, sim in words_sorted[:top_n]:
            print(word, sim)
            
        pass

#--- EXAMPLE RUN --------------------------------------------------------------+

settings = {}
settings['n'] = 5                   # dimension of word embeddings
settings['window_size'] = 2         # context window +/- center word
settings['min_count'] = 0           # minimum word count
settings['epochs'] = 5000           # number of training epochs
settings['neg_samp'] = 10           # number of negative words to use during training
settings['learning_rate'] = 0.01    # learning rate
np.random.seed(0)                   # set the seed for reproducibility

corpus = [['the','quick','brown','fox','jumped','over','the','lazy','dog']]

# INITIALIZE W2V MODEL
w2v = word2vec()

# generate training data
training_data = w2v.generate_training_data(settings, corpus)

# train word2vec model
w2v.train(training_data)

#--- END ----------------------------------------------------------------------+

EPOCH: 0 LOSS: 115.711946046
EPOCH: 1 LOSS: 114.736293974
EPOCH: 2 LOSS: 113.832568632
EPOCH: 3 LOSS: 112.992428501
EPOCH: 4 LOSS: 112.208726729
EPOCH: 5 LOSS: 111.47530631
EPOCH: 6 LOSS: 110.786835298
EPOCH: 7 LOSS: 110.138673405
EPOCH: 8 LOSS: 109.526763334
EPOCH: 9 LOSS: 108.947541748
EPOCH: 10 LOSS: 108.397865917
EPOCH: 11 LOSS: 107.874952963
EPOCH: 12 LOSS: 107.376329274
EPOCH: 13 LOSS: 106.899788188
EPOCH: 14 LOSS: 106.443354426
EPOCH: 15 LOSS: 106.005254059
EPOCH: 16 LOSS: 105.583889059
EPOCH: 17 LOSS: 105.177815627
EPOCH: 18 LOSS: 104.785725681
EPOCH: 19 LOSS: 104.406430981
EPOCH: 20 LOSS: 104.038849466
EPOCH: 21 LOSS: 103.681993468
EPOCH: 22 LOSS: 103.334959495
EPOCH: 23 LOSS: 102.99691937
EPOCH: 24 LOSS: 102.667112505
EPOCH: 25 LOSS: 102.344839165
EPOCH: 26 LOSS: 102.029454568
EPOCH: 27 LOSS: 101.720363708
EPOCH: 28 LOSS: 101.417016813
EPOCH: 29 LOSS: 101.118905338
EPOCH: 30 LOSS: 100.825558431
EPOCH: 31 LOSS: 100.536539809
EPOCH: 32 LOSS: 100.251444996
EPOCH: 33 LOSS: 99.969

EPOCH: 315 LOSS: 63.7835594981
EPOCH: 316 LOSS: 63.7672715202
EPOCH: 317 LOSS: 63.7511613406
EPOCH: 318 LOSS: 63.7352262908
EPOCH: 319 LOSS: 63.7194637533
EPOCH: 320 LOSS: 63.7038711602
EPOCH: 321 LOSS: 63.6884459925
EPOCH: 322 LOSS: 63.6731857784
EPOCH: 323 LOSS: 63.6580880927
EPOCH: 324 LOSS: 63.6431505554
EPOCH: 325 LOSS: 63.6283708305
EPOCH: 326 LOSS: 63.6137466255
EPOCH: 327 LOSS: 63.5992756899
EPOCH: 328 LOSS: 63.5849558143
EPOCH: 329 LOSS: 63.5707848299
EPOCH: 330 LOSS: 63.5567606069
EPOCH: 331 LOSS: 63.5428810539
EPOCH: 332 LOSS: 63.5291441172
EPOCH: 333 LOSS: 63.5155477798
EPOCH: 334 LOSS: 63.5020900604
EPOCH: 335 LOSS: 63.4887690129
EPOCH: 336 LOSS: 63.4755827253
EPOCH: 337 LOSS: 63.4625293191
EPOCH: 338 LOSS: 63.4496069488
EPOCH: 339 LOSS: 63.4368138007
EPOCH: 340 LOSS: 63.4241480923
EPOCH: 341 LOSS: 63.4116080719
EPOCH: 342 LOSS: 63.3991920178
EPOCH: 343 LOSS: 63.3868982374
EPOCH: 344 LOSS: 63.3747250669
EPOCH: 345 LOSS: 63.3626708706
EPOCH: 346 LOSS: 63.3507340401
EPOCH: 3

EPOCH: 584 LOSS: 62.0455723137
EPOCH: 585 LOSS: 62.0429015027
EPOCH: 586 LOSS: 62.0402406064
EPOCH: 587 LOSS: 62.0375895617
EPOCH: 588 LOSS: 62.0349483062
EPOCH: 589 LOSS: 62.0323167782
EPOCH: 590 LOSS: 62.0296949173
EPOCH: 591 LOSS: 62.0270826634
EPOCH: 592 LOSS: 62.0244799575
EPOCH: 593 LOSS: 62.0218867414
EPOCH: 594 LOSS: 62.0193029577
EPOCH: 595 LOSS: 62.0167285495
EPOCH: 596 LOSS: 62.0141634611
EPOCH: 597 LOSS: 62.0116076373
EPOCH: 598 LOSS: 62.0090610236
EPOCH: 599 LOSS: 62.0065235665
EPOCH: 600 LOSS: 62.0039952129
EPOCH: 601 LOSS: 62.0014759108
EPOCH: 602 LOSS: 61.9989656086
EPOCH: 603 LOSS: 61.9964642556
EPOCH: 604 LOSS: 61.9939718016
EPOCH: 605 LOSS: 61.9914881973
EPOCH: 606 LOSS: 61.9890133938
EPOCH: 607 LOSS: 61.9865473433
EPOCH: 608 LOSS: 61.9840899982
EPOCH: 609 LOSS: 61.9816413118
EPOCH: 610 LOSS: 61.9792012379
EPOCH: 611 LOSS: 61.9767697312
EPOCH: 612 LOSS: 61.9743467467
EPOCH: 613 LOSS: 61.9719322401
EPOCH: 614 LOSS: 61.9695261679
EPOCH: 615 LOSS: 61.9671284869
EPOCH: 6

EPOCH: 865 LOSS: 61.567235392
EPOCH: 866 LOSS: 61.5662565204
EPOCH: 867 LOSS: 61.5652813295
EPOCH: 868 LOSS: 61.5643098046
EPOCH: 869 LOSS: 61.5633419311
EPOCH: 870 LOSS: 61.5623776944
EPOCH: 871 LOSS: 61.5614170799
EPOCH: 872 LOSS: 61.5604600729
EPOCH: 873 LOSS: 61.559506659
EPOCH: 874 LOSS: 61.5585568236
EPOCH: 875 LOSS: 61.5576105522
EPOCH: 876 LOSS: 61.5566678303
EPOCH: 877 LOSS: 61.5557286434
EPOCH: 878 LOSS: 61.5547929772
EPOCH: 879 LOSS: 61.5538608173
EPOCH: 880 LOSS: 61.5529321492
EPOCH: 881 LOSS: 61.5520069586
EPOCH: 882 LOSS: 61.5510852312
EPOCH: 883 LOSS: 61.5501669527
EPOCH: 884 LOSS: 61.5492521088
EPOCH: 885 LOSS: 61.5483406853
EPOCH: 886 LOSS: 61.5474326681
EPOCH: 887 LOSS: 61.5465280428
EPOCH: 888 LOSS: 61.5456267955
EPOCH: 889 LOSS: 61.5447289119
EPOCH: 890 LOSS: 61.5438343781
EPOCH: 891 LOSS: 61.5429431798
EPOCH: 892 LOSS: 61.5420553032
EPOCH: 893 LOSS: 61.5411707343
EPOCH: 894 LOSS: 61.540289459
EPOCH: 895 LOSS: 61.5394114636
EPOCH: 896 LOSS: 61.538536734
EPOCH: 897 L

EPOCH: 1169 LOSS: 61.3841114227
EPOCH: 1170 LOSS: 61.3837515904
EPOCH: 1171 LOSS: 61.3833927263
EPOCH: 1172 LOSS: 61.3830348266
EPOCH: 1173 LOSS: 61.3826778872
EPOCH: 1174 LOSS: 61.3823219043
EPOCH: 1175 LOSS: 61.381966874
EPOCH: 1176 LOSS: 61.3816127924
EPOCH: 1177 LOSS: 61.3812596556
EPOCH: 1178 LOSS: 61.3809074598
EPOCH: 1179 LOSS: 61.3805562012
EPOCH: 1180 LOSS: 61.380205876
EPOCH: 1181 LOSS: 61.3798564804
EPOCH: 1182 LOSS: 61.3795080107
EPOCH: 1183 LOSS: 61.379160463
EPOCH: 1184 LOSS: 61.3788138338
EPOCH: 1185 LOSS: 61.3784681192
EPOCH: 1186 LOSS: 61.3781233157
EPOCH: 1187 LOSS: 61.3777794194
EPOCH: 1188 LOSS: 61.3774364269
EPOCH: 1189 LOSS: 61.3770943344
EPOCH: 1190 LOSS: 61.3767531383
EPOCH: 1191 LOSS: 61.376412835
EPOCH: 1192 LOSS: 61.3760734211
EPOCH: 1193 LOSS: 61.3757348928
EPOCH: 1194 LOSS: 61.3753972467
EPOCH: 1195 LOSS: 61.3750604792
EPOCH: 1196 LOSS: 61.3747245868
EPOCH: 1197 LOSS: 61.3743895661
EPOCH: 1198 LOSS: 61.3740554136
EPOCH: 1199 LOSS: 61.3737221257
EPOCH: 1200 

...

EPOCH: 3519 LOSS: 61.149530845
EPOCH: 3520 LOSS: 61.1494896154
EPOCH: 3521 LOSS: 61.1494483987
EPOCH: 3522 LOSS: 61.1494071949
EPOCH: 3523 LOSS: 61.1493660039
EPOCH: 3524 LOSS: 61.1493248258
EPOCH: 3525 LOSS: 61.1492836606
EPOCH: 3526 LOSS: 61.1492425082
EPOCH: 3527 LOSS: 61.1492013686
EPOCH: 3528 LOSS: 61.1491602418
EPOCH: 3529 LOSS: 61.1491191278
EPOCH: 3530 LOSS: 61.1490780266
EPOCH: 3531 LOSS: 61.1490369383
EPOCH: 3532 LOSS: 61.1489958627
EPOCH: 3533 LOSS: 61.1489547998
EPOCH: 3534 LOSS: 61.1489137498
EPOCH: 3535 LOSS: 61.1488727125
EPOCH: 3536 LOSS: 61.1488316879
EPOCH: 3537 LOSS: 61.1487906761
EPOCH: 3538 LOSS: 61.148749677
EPOCH: 3539 LOSS: 61.1487086906
EPOCH: 3540 LOSS: 61.1486677169
EPOCH: 3541 LOSS: 61.1486267559
EPOCH: 3542 LOSS: 61.1485858077
EPOCH: 3543 LOSS: 61.148544872
EPOCH: 3544 LOSS: 61.1485039491
EPOCH: 3545 LOSS: 61.1484630388
EPOCH: 3546 LOSS: 61.1484221412
EPOCH: 3547 LOSS: 61.1483812562
EPOCH: 3548 LOSS: 61.1483403839
EPOCH: 3549 LOSS: 61.1482995242
EPOCH: 3550

EPOCH: 3830 LOSS: 61.1372871652
EPOCH: 3831 LOSS: 61.1372495441
EPOCH: 3832 LOSS: 61.1372119335
EPOCH: 3833 LOSS: 61.1371743334
EPOCH: 3834 LOSS: 61.1371367437
EPOCH: 3835 LOSS: 61.1370991645
EPOCH: 3836 LOSS: 61.1370615957
EPOCH: 3837 LOSS: 61.1370240374
EPOCH: 3838 LOSS: 61.1369864894
EPOCH: 3839 LOSS: 61.136948952
EPOCH: 3840 LOSS: 61.1369114249
EPOCH: 3841 LOSS: 61.1368739083
EPOCH: 3842 LOSS: 61.136836402
EPOCH: 3843 LOSS: 61.1367989062
EPOCH: 3844 LOSS: 61.1367614207
EPOCH: 3845 LOSS: 61.1367239457
EPOCH: 3846 LOSS: 61.136686481
EPOCH: 3847 LOSS: 61.1366490267
EPOCH: 3848 LOSS: 61.1366115827
EPOCH: 3849 LOSS: 61.1365741492
EPOCH: 3850 LOSS: 61.136536726
EPOCH: 3851 LOSS: 61.1364993131
EPOCH: 3852 LOSS: 61.1364619106
EPOCH: 3853 LOSS: 61.1364245184
EPOCH: 3854 LOSS: 61.1363871365
EPOCH: 3855 LOSS: 61.136349765
EPOCH: 3856 LOSS: 61.1363124038
EPOCH: 3857 LOSS: 61.1362750529
EPOCH: 3858 LOSS: 61.1362377123
EPOCH: 3859 LOSS: 61.136200382
EPOCH: 3860 LOSS: 61.1361630621
EPOCH: 3861 LO

EPOCH: 4145 LOSS: 61.1259241385
EPOCH: 4146 LOSS: 61.1258895341
EPOCH: 4147 LOSS: 61.1258549384
EPOCH: 4148 LOSS: 61.1258203516
EPOCH: 4149 LOSS: 61.1257857734
EPOCH: 4150 LOSS: 61.1257512041
EPOCH: 4151 LOSS: 61.1257166435
EPOCH: 4152 LOSS: 61.1256820916
EPOCH: 4153 LOSS: 61.1256475485
EPOCH: 4154 LOSS: 61.1256130142
EPOCH: 4155 LOSS: 61.1255784885
EPOCH: 4156 LOSS: 61.1255439716
EPOCH: 4157 LOSS: 61.1255094635
EPOCH: 4158 LOSS: 61.125474964
EPOCH: 4159 LOSS: 61.1254404733
EPOCH: 4160 LOSS: 61.1254059913
EPOCH: 4161 LOSS: 61.125371518
EPOCH: 4162 LOSS: 61.1253370534
EPOCH: 4163 LOSS: 61.1253025976
EPOCH: 4164 LOSS: 61.1252681504
EPOCH: 4165 LOSS: 61.1252337119
EPOCH: 4166 LOSS: 61.1251992821
EPOCH: 4167 LOSS: 61.1251648609
EPOCH: 4168 LOSS: 61.1251304485
EPOCH: 4169 LOSS: 61.1250960447
EPOCH: 4170 LOSS: 61.1250616496
EPOCH: 4171 LOSS: 61.1250272632
EPOCH: 4172 LOSS: 61.1249928854
EPOCH: 4173 LOSS: 61.1249585163
EPOCH: 4174 LOSS: 61.1249241559
EPOCH: 4175 LOSS: 61.124889804
EPOCH: 4176

EPOCH: 4425 LOSS: 61.116562229
EPOCH: 4426 LOSS: 61.1165299189
EPOCH: 4427 LOSS: 61.1164976164
EPOCH: 4428 LOSS: 61.1164653216
EPOCH: 4429 LOSS: 61.1164330344
EPOCH: 4430 LOSS: 61.1164007549
EPOCH: 4431 LOSS: 61.1163684831
EPOCH: 4432 LOSS: 61.1163362188
EPOCH: 4433 LOSS: 61.1163039623
EPOCH: 4434 LOSS: 61.1162717133
EPOCH: 4435 LOSS: 61.116239472
EPOCH: 4436 LOSS: 61.1162072383
EPOCH: 4437 LOSS: 61.1161750123
EPOCH: 4438 LOSS: 61.1161427938
EPOCH: 4439 LOSS: 61.116110583
EPOCH: 4440 LOSS: 61.1160783798
EPOCH: 4441 LOSS: 61.1160461842
EPOCH: 4442 LOSS: 61.1160139962
EPOCH: 4443 LOSS: 61.1159818158
EPOCH: 4444 LOSS: 61.115949643
EPOCH: 4445 LOSS: 61.1159174778
EPOCH: 4446 LOSS: 61.1158853202
EPOCH: 4447 LOSS: 61.1158531702
EPOCH: 4448 LOSS: 61.1158210278
EPOCH: 4449 LOSS: 61.1157888929
EPOCH: 4450 LOSS: 61.1157567657
EPOCH: 4451 LOSS: 61.115724646
EPOCH: 4452 LOSS: 61.1156925339
EPOCH: 4453 LOSS: 61.1156604293
EPOCH: 4454 LOSS: 61.1156283323
EPOCH: 4455 LOSS: 61.1155962429
EPOCH: 4456 L

EPOCH: 4715 LOSS: 61.1075000269
EPOCH: 4716 LOSS: 61.1074698029
EPOCH: 4717 LOSS: 61.1074395857
EPOCH: 4718 LOSS: 61.1074093752
EPOCH: 4719 LOSS: 61.1073791715
EPOCH: 4720 LOSS: 61.1073489745
EPOCH: 4721 LOSS: 61.1073187843
EPOCH: 4722 LOSS: 61.1072886008
EPOCH: 4723 LOSS: 61.1072584241
EPOCH: 4724 LOSS: 61.1072282541
EPOCH: 4725 LOSS: 61.1071980908
EPOCH: 4726 LOSS: 61.1071679343
EPOCH: 4727 LOSS: 61.1071377845
EPOCH: 4728 LOSS: 61.1071076415
EPOCH: 4729 LOSS: 61.1070775051
EPOCH: 4730 LOSS: 61.1070473755
EPOCH: 4731 LOSS: 61.1070172527
EPOCH: 4732 LOSS: 61.1069871365
EPOCH: 4733 LOSS: 61.1069570271
EPOCH: 4734 LOSS: 61.1069269243
EPOCH: 4735 LOSS: 61.1068968283
EPOCH: 4736 LOSS: 61.106866739
EPOCH: 4737 LOSS: 61.1068366564
EPOCH: 4738 LOSS: 61.1068065805
EPOCH: 4739 LOSS: 61.1067765113
EPOCH: 4740 LOSS: 61.1067464488
EPOCH: 4741 LOSS: 61.106716393
EPOCH: 4742 LOSS: 61.1066863439
EPOCH: 4743 LOSS: 61.1066563015
EPOCH: 4744 LOSS: 61.1066262657
EPOCH: 4745 LOSS: 61.1065962367
EPOCH: 474